# AgentKit SDK 教程 00: 配置与凭证

欢迎来到 AgentKit SDK！本教程是所有新用户的**第一站**，将引导你完成 AgentKit 的环境配置。

在本 Notebook 中，你将学会：

1. 理解 AgentKit 中的两种关键凭证（环境变量）；
2. 如何为 SDK 与 CLI 配置云服务访问凭证；
3. 如何为 Agent 运行时配置模型访问等环境变量。

完成本教程后，你的开发环境将准备就绪，可以顺利运行其他所有教程。

## 1. 两种关键凭证及其用途

在 AgentKit 中，我们主要和两类环境变量（凭证）打交道，清晰地理解它们的区别非常重要：

### 第一类：SDK 与 CLI 的云服务访问凭证

- **用途**：当你在本地（比如本 Notebook 或终端）运行 `agentkit` 命令或调用 AgentKit SDK 时，需要用这类凭证来**授权你访问火山引擎的云服务**。这些操作包括创建/管理镜像仓库（CR）、持续集成流水线（CodePipeline）、对象存储（TOS）以及部署 Agent 所需的 Runtime 资源。
- **变量名**：`VOLCENGINE_ACCESS_KEY`, `VOLCENGINE_SECRET_KEY`, `VOLCENGINE_REGION`。
- **配置场景**：主要用于本地开发环境，是你作为“开发者”的身份凭证。

### 第二类：Agent 运行时的环境变量

- **用途**：当你的 Agent **代码在云上（Runtime）或本地（Docker）运行时**，需要用这类凭证来**访问外部服务**，最常见的就是访问方舟（Ark）大模型。
- **变量名**：例如 `MODEL_AGENT_NAME`, `MODEL_AGENT_API_KEY` 等，具体变量名取决于你的 Agent 代码实现。
- **配置场景**：这些变量需要被注入到 Agent 的运行环境中（容器），是你“Agent”的身份凭证。

简单来说：第一类是你开发时用的，第二类是你的 Agent 运行时用的。下面我们分别介绍如何配置它们。

## 2. 配置 SDK 与 CLI 的云服务访问凭证

我们推荐使用 `agentkit` 命令行工具来全局管理你的云服务凭证。这种方式最安全、便捷，且一次配置，到处生效。

### 推荐方式：使用 `agentkit config` 命令

打开你的终端，执行以下命令，将 `YOUR_ACCESS_KEY` 和 `YOUR_SECRET_KEY` 替换为你的真实凭证：

```bash
agentkit config --global --set volcengine.access_key=YOUR_ACCESS_KEY
agentkit config --global --set volcengine.secret_key=YOUR_SECRET_KEY
agentkit config --global --set volcengine.region=cn-beijing  # 或其他区域
```

执行完毕后，AgentKit 会将你的凭证安全地存储在全局配置文件 `~/.agentkit/config.yaml` 中。SDK 和 CLI 会自动读取，无需再进行其他操作。

### 备选方式：使用环境变量

如果你不希望创建全局配置文件，也可以在运行 SDK/CLI 的终端中设置环境变量。

```bash
export VOLCENGINE_ACCESS_KEY="YOUR_ACCESS_KEY"
export VOLCENGINE_SECRET_KEY="YOUR_SECRET_KEY"
export VOLCENGINE_REGION="cn-beijing"
```

> **注意**：这种方式只在当前终端会话中有效。如果打开新的终端，需要重新设置。

## 3. 配置 Agent 运行时的环境变量

Agent 运行时的环境变量通过项目配置文件 `agentkit.yaml` 中的 `runtime_envs` 字段来管理。这样可以确保每次部署时，正确的环境变量都会被注入到你的 Agent 运行环境中。

### 配置方式：修改 `agentkit.yaml`

当你使用 `sdk.init_project` 创建一个新项目后，会得到一个 `agentkit.yaml` 文件。你可以通过 `AgentConfig` 类来以编程方式修改它。

下面是一个示例，演示如何为一个 Agent 项目添加访问方舟模型所需的环境变量：


In [ ]:
from pathlib import Path
from agentkit.toolkit import sdk
from agentkit.toolkit.sdk import AgentConfig

# 1. 准备一个示例项目（如果不存在）
project_root = Path("tutorial_projects/t00-config-demo")
if not project_root.exists():
    sdk.init_project(project_name=project_root.name, template="basic", project_root=str(project_root))
    print(f"示例项目已创建在: {project_root.resolve()}")
else:
    print(f"示例项目已存在: {project_root.resolve()}")

# 2. 加载项目配置
config = AgentConfig.load(project_root)

# 3. 添加或更新运行时环境变量
config.add_runtime_env("MODEL_AGENT_NAME", "your-model-agent-name")
config.add_runtime_env("MODEL_AGENT_API_KEY", "your-model-agent-api-key")

# 4. 保存配置
config.save()

print("已成功向 agentkit.yaml 中写入运行时环境变量:")
print(config.runtime_envs)

> **提示**：
> - `your-model-agent-name` 和 `your-model-agent-api-key` 需要替换为你的模型终端节点和对应的 API Key。
> - 当你的 Agent 部署在**云上 Runtime** 并使用 veADK 时，AgentKit 会自动为你注入访问火山引擎服务的临时 AK/SK，因此你**不需要**在 `runtime_envs` 中手动配置 `VOLCENGINE_*` 变量。
> - 当你的 Agent 在**本地 Docker** 中运行时，如果 Agent 代码需要访问火山引擎服务（例如，调用 veadk），则建议将第一类凭证也添加到 `runtime_envs` 中。

### 如何在本地运行的 Docker 中传递凭证？

如果你使用 `local` 策略在本地 Docker 中运行 Agent，并且你的 Agent 代码需要访问火山引擎云服务，你可以这样做：


In [ ]:
import os

# 假设你的 AK/SK 已经配置在环境变量中
ak = os.environ.get("VOLCENGINE_ACCESS_KEY")
sk = os.environ.get("VOLCENGINE_SECRET_KEY")
region = os.environ.get("VOLCENGINE_REGION", "cn-beijing")

if ak and sk:
    config = AgentConfig.load(project_root)
    
    # 获取 local 策略的配置，并为其添加环境变量
    local_strategy_config = config.get_strategy_config("local") or {}
    runtime_envs = local_strategy_config.get("runtime_envs", {})
    runtime_envs.update({
        "VOLCENGINE_ACCESS_KEY": ak,
        "VOLCENGINE_SECRET_KEY": sk,
        "VOLCENGINE_REGION": region
    })
    
    config.update_strategy_config({"runtime_envs": runtime_envs}, strategy_name="local")
    config.save()
    
    print("已将本地 AK/SK 写入 local 策略的 runtime_envs 中。")
else:
    print("未在环境变量中找到 VOLCENGINE_ACCESS_KEY 或 VOLCENGINE_SECRET_KEY。")
    print("如果你需要在本地 Docker 中访问云服务，请先配置好第一类凭证。")

## 总结

恭喜你！现在你已经掌握了 AgentKit 中两类凭证的配置方法。

- **SDK/CLI 凭证**：通过 `~/.agentkit/config.yaml` 或环境变量配置，用于开发。
- **Agent 运行时凭证**：通过 `agentkit.yaml` 的 `runtime_envs` 配置，用于 Agent 运行。

有了这些知识，你就可以顺利地进行后续的 Agent 开发与部署了。现在，请继续学习其他教程！